In [3]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [4]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

from numpy.random import seed
seed(1)

#from tensorflow import set_random_seed
#set_random_seed(2)

In [5]:
batch_size = 64  # Batch size for training.
epochs = 5  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 29976  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'prim_fwd/minialpha.train'

## The data
The data is present in a tab delimited CSV file. To give it a quick overview we will load it with pandas:

In [6]:
df = pd.read_csv(data_path,delimiter='\t')

In [7]:
df.head()

,BfkEwEd2,kurd1d2wEd2urd1d3wEd3
0,BfwEd3,urd1dawEda
1,BfkEwEda,kurd1d2wEd2urd1dawEda
2,Bfkud2EsE,kwEd2sE
3,BfkEwEe1,kurd1d2wEd2tE
4,BfkEuEyE,kurd1d2wEd2kue1uEpEyE


In [8]:
del df # Save memory

## Vectorizing data

In [9]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

# Loop over lines
lines = open(data_path).read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    # Input and target are split by tabs
    input_text, target_text = line.split('\t')
    
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    # Create a set of all unique characters in the input
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
            
    # Create a set of all unique output characters
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

print('Number of samples:', len(input_texts))

Number of samples: 29976


In [10]:
input_characters = sorted(list(input_characters)) # Make sure we achieve the same order in our input chars
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters) 
num_decoder_tokens = len(target_characters) 


print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)

Number of unique input tokens: 32
Number of unique output tokens: 34


In [11]:
# This works very similar to a tokenizer
# The index maps a character to a number
input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}

In [12]:
max_encoder_seq_length = max([len(txt) for txt in input_texts]) # Get longest sequences length
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Max sequence length for inputs: 74
Max sequence length for outputs: 508


In [13]:

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')


decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

# decoder_target_data is the same as decoder_input_data but offset by one timestep. 
# decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :]

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [14]:
# Loop over input texts
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # Loop over each char in an input text
    for t, char in enumerate(input_text):
        # Create one hot encoding by setting the index to 1
        encoder_input_data[i, t, input_token_index[char]] = 1.
    # Loop over each char in the output text
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [15]:

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                       name = 'encoder_inputs')

# The return_state contructor argument, configuring a RNN layer to return a list 
# where the first entry is the outputs and the next entries are the internal RNN states. 
# This is used to recover the states of the encoder.
encoder = LSTM(latent_dim, 
                    return_state=True, 
                    name = 'encoder')

encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens), 
                       name = 'decoder_inputs')

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, 
                         return_sequences=True, 
                         return_state=True, 
                         name = 'decoder_lstm')

# The inital_state call argument, specifying the initial state(s) of a RNN. 
# This is used to pass the encoder states to the decoder as initial states.
# Basically making the first memory of the decoder the encoded semantics
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, 
                      activation='softmax', 
                      name = 'decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [16]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
history = model.fit([encoder_input_data, decoder_input_data], 
                    decoder_target_data,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)
# Save model
#model.save('s2s.h5')

Epoch 1/5
375/375 [==============================] - 1983s 5s/step - loss: 0.2052 - val_loss: 0.1270
Epoch 2/5
375/375 [==============================] - 1886s 5s/step - loss: 0.1813 - val_loss: 0.1003
Epoch 3/5
375/375 [==============================] - 2145s 6s/step - loss: 0.1552 - val_loss: 0.0888
Epoch 4/5
375/375 [==============================] - 3068s 8s/step - loss: 0.1415 - val_loss: 0.0817
Epoch 5/5
375/375 [==============================] - 2031s 5s/step - loss: 0.1284 - val_loss: 0.0756


# Creating inference models

In [17]:
# Define encoder model
encoder_model = Model(encoder_inputs, encoder_states)

In [18]:
# Define decoder model

# Inputs from the encoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))

# Create a combined memory to input into the decoder
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Decoder
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

# Predict next char
decoder_outputs = decoder_dense(decoder_outputs)

# The model takes in the encoder memory plus it's own memory as an input and spits out 
# a prediction plus its own memory to be used for the next char
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [19]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = {i: char 
                            for char, i in input_token_index.items()}
reverse_target_char_index = {i: char 
                             for char, i in target_token_index.items()}

In [20]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    # Loop untill we recieve a stop sign
    while not stop_condition:
        # Get output and internal states of the decoder 
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Get the predicted token (the token with the highest score)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # Get the character belonging to the token
        sampled_char = reverse_target_char_index[sampled_token_index]
        # Append char to output
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

## Giving it a spin


In [39]:
my_text = 'BfkEob'
placeholder = np.zeros((1,len(my_text)+10,num_encoder_tokens))

In [40]:
for i, char in enumerate(my_text):
    print(i,char, input_token_index[char])
    placeholder[0,i,input_token_index[char]] = 1

0 B 5
1 f 13
2 k 17
3 E 8
4 o 21
5 b 10


In [79]:
decode_sequence(placeholder) #correct answer is kurd1d2wEd2uEob so kinda close but not exactly.

'ure1dawEda\n'

In [ ]:
decode_sequence()

In [84]:
my_text = 'BfuEkeaE'
placeholder = np.zeros((1,len(my_text)+10,num_encoder_tokens))

In [85]:
for i, char in enumerate(my_text):
    print(i,char, input_token_index[char])
    placeholder[0,i,input_token_index[char]] = 1

0 B 5
1 f 13
2 u 27
3 E 8
4 k 17
5 e 12
6 a 9
7 E 8


In [86]:
decode_sequence(placeholder)

'kure1dawEdaurd1dawEda\n'

In [21]:
def lettoword(let):
    my_dict = {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9','4':'a', 'E': 'b', 'I': 'c', '(+)': 'd', '(-)': 'e', "Y'": 'f', 'abs': 'g', 'acos': 'h', 'acosh': 'i', '+': 'k', 'asin': 'l', 'asinh': 'm', 'atan': 'n', 'atanh': 'o', 'cos': 'p', 'cosh': 'q', '/': 'r', 'exp': 's', 'ln': 't', '*': 'u', 'pi': 'v', 'pow': 'w', 'sign': 'x', 'sin': 'y', 'sinh': 'z', 'sqrt': 'A', '-': 'B', 'tan': 'C', 'tanh': 'D', 'x': 'E'}
    rev = {v:k for k, v in my_dict.items()}
    let = list(let)
    ret = []
    for key in let:
        ret.append(rev[key])
    return " ".join(ret)
        
def wordtolet(word):
    my_dict = {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '4':'a', 'E': 'b', 'I': 'c', '(+)': 'd', '(-)': 'e', "Y'": 'f', 'abs': 'g', 'acos': 'h', 'acosh': 'i', '+': 'k', 'asin': 'l', 'asinh': 'm', 'atan': 'n', 'atanh': 'o', 'cos': 'p', 'cosh': 'q', '/': 'r', 'exp': 's', 'ln': 't', '*': 'u', 'pi': 'v', 'pow': 'w', 'sign': 'x', 'sin': 'y', 'sinh': 'z', 'sqrt': 'A', '-': 'B', 'tan': 'C', 'tanh': 'D', 'x': 'E'}
    word = word.split()
    ret = []
    for key in word:
        ret.append(my_dict[key])
    return "".join(ret)

In [22]:
def solve(text):
    my_text = wordtolet(text)
    placeholder = np.zeros((1,len(my_text)+10,num_encoder_tokens))
    for i, char in enumerate(my_text):
        #print(i,char, input_token_index[char])
        placeholder[0,i,input_token_index[char]] = 1
    sol = decode_sequence(placeholder)[:-1]
    sol = lettoword(sol)
    return sol


In [30]:
def calc_accuracy():
    test_lines = open('alpha_integ.test').read().split('\n')
    
        
    tot = len(input_texts)
    corr = 0
    for test_line in test_lines:
        test_input_eq, test_target_eq = test_line.split('\t')
        placeholder = np.zeros((1,len(test_input_eq)+10,num_encoder_tokens))
        for i, char in enumerate(test_input_eq):
            #print(i,char, input_token_index[char])
            placeholder[0,i,input_token_index[char]] = 1
        if(decode_sequence(placeholder)[:-1] == test_target_eq):
            print(test_input_eq, test_target_eq)
            corr += 1
    return (corr/tot)*100

In [25]:
def print_examples():
    train_lines = open('prim_fwd/minialpha.train').read().split('\n')
    for train_line in train_lines[:1]:
        train_input_eq, train_target_eq = train_line.split('\t')
        train_input_eq = lettoword(train_input_eq)
        train_target_eq = lettoword(train_target_eq)
        sol = solve(train_input_eq)
        #if(solve(train_input_eq) == train_target_eq):
        print("\nInput: " + train_input_eq + "\nPrediction: " + lettoword(wordtolet(sol)[:min(len(wordtolet(sol)), len(wordtolet(train_target_eq)))]) + "\nTarget: " + train_target_eq)
        

In [146]:
print_examples()


Input: - Y' + x pow x (+) 2
Prediction: + * / (-) 1 (+) 4 pow x (+) 4 * / (+) 1 (+) 4 pow x (+) 4
Target: + * / (+) 1 (+) 2 pow x (+) 2 * / (+) 1 (+) 3 pow x (+) 3

Input: - Y' pow x (+) 3
Prediction: * / (-) 1 (+) 4 pow x (+) 4
Target: * / (+) 1 (+) 4 pow x (+) 4

Input: - Y' + x pow x (+) 4
Prediction: + * / (-) 1 (+) 4 pow x (+) 4 * / (+) 1 (+) 4 pow x (+) 4
Target: + * / (+) 1 (+) 2 pow x (+) 2 * / (+) 1 (+) 4 pow x (+) 4


In [26]:
print_examples()


Input: - Y' + x pow x (+) 2
Prediction: + * / (+) 1 (+) 2 pow x (+) 2 + * / (+) 1 (+) 4 pow x (+)
Target: + * / (+) 1 (+) 2 pow x (+) 2 * / (+) 1 (+) 3 pow x (+) 3


In [37]:
encoder_model.save('int_enc.tf', save_format = 'tf')

INFO:tensorflow:Assets written to: int_enc.tf\assets


INFO:tensorflow:Assets written to: int_enc.tf\assets


In [38]:
decoder_model.save('int_dec.tf', save_format = 'tf')

INFO:tensorflow:Assets written to: int_dec.tf\assets


INFO:tensorflow:Assets written to: int_dec.tf\assets
